# Project 1: Boolean Satisfiability

In [1]:
import json
from pathlib import Path
from argparse import ArgumentParser
from dimacs_parser import DimacsParser
from model_timer import Timer

import numpy as np

input_file = '../toy_tester.cnf'
path = Path(input_file)
filename = path.name
instance = DimacsParser.parse_cnf_file(input_file)
print(instance, end="")

Number of variables: 2
Number of clauses: 3
Variables: {1, 2}
Clause 0: {1}
Clause 1: {2, -1}
Clause 2: {-2}


In [2]:
symbols = list(instance.vars)
clauses = [list(clause) for clause in instance.clauses]
model = {}

In [3]:
def find_prop_unit_clauses(clauses, model):
    while(1):
        unit_run = False
        
        for clause in clauses: 
            ex_model = [x for x in clause if abs(x) not in model]    
            
            if (len(ex_model) == 1):
                model[abs(ex_model[0])] = ex_model[0] > 0
                unit_run = True
        
        if unit_run == False: 
            break
    return model
        

In [4]:
def eval_clause(clause, model):
    eval = {}
    for var in clause:
        if abs(var) in model: 
            if var > 0 and model[var]:
                eval[var] = 'T' 
            elif var < 0 and not model[abs(var)]: 
                eval[var] = 'T'
            else: 
                eval[var] = 'F'
        else: 
            eval[var] = 'U'
    if all(value=='F' for value in eval.values()):
        res = 'FALSE'
    elif any(value=='T' for value in eval.values()):
        res = 'TRUE'
    else:
        res = 'UNKNOWN'
    return res

In [5]:
def eval_instance(clauses, model): 
    if all(eval_clause(clause, model)=='TRUE' for clause in clauses): 
        return 'SAT' 
    elif any(eval_clause(clause, model)=='FALSE' for clause in clauses):
        return 'UNSAT'
    else: 
        return 'UNKNOWN'

In [6]:
# pure symbol 
def pure_symbol(symbols, clauses, model):
    pure = {} 
    impure = []
    for clause in clauses: 
        if eval_clause(clause, model) is True: 
            continue 

        for x in clause: 
            var = abs(x)

            if var in model or var in impure:
                continue

            if var not in pure:
                    pure[var] = x > 0

            elif var in pure: 
                if pure[var] != (x > 0): 
                    pure.pop(var)
                    impure.append(var)
    if not pure: 
        return [], []
    return list(pure.keys()), list(pure.values())

In [7]:
# unit clause 
def unit_clause(clauses, model): 
    model = model.copy() 
    cont = 1 

    while cont: 
        cont = 0 

        for clause in clauses:
            if eval_clause(clause, model) is True:
                continue 
            
            unassigned = [x for x in clause if abs(x) not in model]

            if len(unassigned) == 1: 
                var = unassigned[0]
                if (abs(var)) in model: 
                    continue 
                else: 
                    model[abs(var)] = var > 0

    return list(model.keys()), list(model.values())

In [8]:
def extend(model, syms, vals): 
    d1 = dict(zip(syms, vals))
    return model.update(d1)

In [9]:
def branch():
    return True

In [28]:
def dpll(clauses, symbols, model): 
    print('new dpll call')
    if eval_instance(clauses, model)=='SAT':
        print('sat')
        return True 
    elif eval_instance(clauses, model)=='UNSAT':
        print('unsat')
        return False 
    else: 
        # pure symbol
        print('pure symbol')
        vars, vals = pure_symbol(symbols, clauses, model) 
        if vars:
            symbols = list(set(symbols) - set(vars))
            model = model | dict(zip(vars, vals))
            dpll(clauses, symbols, model)
        else: 
            # unit clause 
            print('unit clause')
            vars, vals = unit_clause(clauses, model)
            if vars: 
                symbols = list(set(symbols) - set(vars))
                model = model | dict(zip(vars, vals))
                dpll(clauses, symbols, model)
            else: 
                # branch
                return False

In [29]:
symbols = list(instance.vars)
clauses = [list(clause) for clause in instance.clauses]
model = {}
clauses, symbols, model

([[1], [2, -1], [-2]], [1, 2], {})

In [30]:
x = dpll(clauses, symbols, model)

new dpll call
pure symbol
unit clause
new dpll call
unsat
